In [764]:
import librosa
import neuro
import random
import numpy as np
import pandas as pd
import risp
import eons
import json
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# original params
if False:
    eo_params = {
        "starting_nodes": 3,
        "starting_edges": 6,
        "merge_rate": 0,
        "population_size": 100,
        "multi_edges": 0,
        "crossover_rate": 0.5,
        "mutation_rate": 0.9,
        "selection_type": "tournament",
        "tournament_size_factor": 0.1,
        "tournament_best_net_factor": 0.9,
        "random_factor": 0.05,
        "num_mutations": 3,
        "node_mutations": { "Threshold": 1.0 },
        "net_mutations": { },
        "edge_mutations": { "Weight": 0.5, "Delay": 0.5 },
        "num_best" : 4
    }

eo_params = {
    "starting_nodes": 3,
    "starting_edges": 6,
    "merge_rate": 0.1,
    "population_size": 100,
    "multi_edges": 0,
    "crossover_rate": 0.5,
    "mutation_rate": .9,
    "selection_type": "tournament",
    "tournament_size_factor": 0.1,
    "tournament_best_net_factor": 0.9,
    "random_factor": 0.05,
    "num_mutations": 20,
    "node_mutations": { "Threshold": 1.0 },
    "net_mutations": { },
    "edge_mutations": { "Weight": 0.5, "Delay": 0.5 },
    "num_best" : 4
}



In [765]:
import numpy as np
import pandas as pd

def generate_sinusoid_scale(pitches=[69], duration=0.5, Fs=4000, amplitude_max=0.5):
    """Generate synthetic sound of scale using sinusoids

    Notebook: C5/C5S1_Scales_CircleFifth.ipynb

    Args:
        pitches (list): List of pitchs (MIDI note numbers) (Default value = [69])
        duration (float): Duration (seconds) (Default value = 0.5)
        Fs (scalar): Sampling rate (Default value = 4000)
        amplitude_max (float): Amplitude (Default value = 0.5)

    Returns:
        x (np.ndarray): Synthesized signal
    """
    N = int(duration * Fs)
    t = np.arange(0, N) / Fs
    x = []
    for p in pitches:
        omega = 2 ** ((p - 69) / 12) * 440
        x = np.append(x, np.sin(2 * np.pi * omega * t))
    x = amplitude_max * x / np.max(x)
    return x

In [766]:
duration = 0.25
Fs = 4000

scale_major = np.array([60, 62, 64, 65, 67, 69, 71, 72])
scale_minor = np.array([57, 59, 60, 62, 64, 65, 67, 69])
scale_major_name = ['C','G','D','A','E','B','F$^\sharp$',
                    'D$^\\flat$','A$^\\flat$','E$^\\flat$','B$^\\flat$','F','C',]
scale_minor_name = ['Am','Em','Bm','F$^\sharp$m','C$^\sharp$m','G$^\sharp$m','D$^\sharp$m',
                    'B$^\\flat$m','Fm','Cm','Gm','Dm','Am',]

scale_major_list = []
for i in range(13):
    x = generate_sinusoid_scale(pitches=scale_major, duration=duration, Fs=Fs)
    scale_major_list.append(x)
    scale_major += 7
    if scale_major[-1] > 80:
        scale_major -= 12
        
scale_minor_list = []
for i in range(13):
    x = generate_sinusoid_scale(pitches=scale_minor, duration=duration, Fs=Fs)
    scale_minor_list.append(x)
    scale_minor += 7
    if scale_minor[-1] > 80:
        scale_minor -= 12

In [767]:
scales_df = pd.DataFrame()
scales_df = scales_df.assign(major_scales=scale_major_name,major_data=scale_major_list,minor_scales=scale_minor_name,minor_data=scale_minor_list)
scales_df


,major_scales,major_data,minor_scales,minor_data
0,C,"[0.0, 0.1997450087080568, 0.3662274977248533, ...",Am,"[0.0, 0.1693689601226457, 0.3187119948743449, ..."
1,G,"[0.0, 0.28878342547725777, 0.47149288186702343...",Em,"[0.0, 0.24747520311908042, 0.4300729133338053,..."
2,D,"[0.0, 0.222550720491966, 0.3985795071729424, 0...",Bm,"[0.0, 0.1891203378067922, 0.3501401517200337, ..."
3,A,"[0.0, 0.1693689601226457, 0.3187119948743449, ...",F$^\sharp$m,"[0.0, 0.2745077478246194, 0.4588738334446349, ..."
4,E,"[0.0, 0.24747520311908042, 0.4300729133338053,...",C$^\sharp$m,"[0.0, 0.2108854056797574, 0.3824206395923214, ..."
5,B,"[0.0, 0.1891203533407224, 0.3501401804797787, ...",G$^\sharp$m,"[0.0, 0.30353148731408547, 0.48240495629789754..."
6,F$^\sharp$,"[0.0, 0.27450784666421385, 0.458873998667307, ...",D$^\sharp$m,"[0.0, 0.23474680969398756, 0.4145327654721112,..."
7,D$^\flat$,"[0.0, 0.2108854230014226, 0.38242067100351496,...",B$^\flat$m,"[0.0, 0.17899960234029763, 0.3342716983525017,..."
8,A$^\flat$,"[0.0, 0.303531438771088, 0.4824048791481337, 0...",Fm,"[0.0, 0.26073229003660775, 0.44495162477587574..."
9,E$^\flat$,"[0.0, 0.23474680969398756, 0.4145327654721112,...",Cm,"[0.0, 0.19974502680422843, 0.36622753090373317..."


In [768]:
def encoder(signal):
    # in this example the signal is a np array 
    
    # note that sample rate must be consistant

    # Extract the MFCC features from the audio signal (this includes mel-shift and more)
    mfcc_features = librosa.feature.mfcc(signal)

    # Preprocess the MFCC features
    scaler = StandardScaler()
    mfcc_features = scaler.fit_transform(mfcc_features)

    # Convert the MFCC features into ROC spikes using Poisson binarization
    roc_spikes = []
    for mfcc_frame in mfcc_features:
        poisson_rate = np.exp(mfcc_frame)
        poisson_spikes = np.random.poisson(poisson_rate)
        roc_spikes.append(poisson_spikes)
    return roc_spikes


In [769]:
spike_df = scales_df
spike_df = spike_df.assign(major_data = spike_df.get('major_data').apply(encoder))
spike_df = spike_df.assign(minor_data = spike_df.get('minor_data').apply(encoder))
spike_df



/var/folders/5q/565vdp5s77g8hjj2qd4j8khm0000gn/T/ipykernel_38616/679666639.py:7: FutureWarning: Pass y=[ 0.          0.19974501  0.3662275  ...  0.23992841 -0.15796156
 -0.45501375] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc_features = librosa.feature.mfcc(signal)
/var/folders/5q/565vdp5s77g8hjj2qd4j8khm0000gn/T/ipykernel_38616/679666639.py:7: FutureWarning: Pass y=[ 0.          0.28878343  0.47149288 ...  0.26864288 -0.30824399
 -0.47383046] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc_features = librosa.feature.mfcc(signal)
/var/folders/5q/565vdp5s77g8hjj2qd4j8khm0000gn/T/ipykernel_38616/679666639.py:7: FutureWarning: Pass y=[ 0.          0.22255072  0.39857951 ...  0.31412243 -0.12044359
 -0.45956274] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc_features = librosa.feature.mfcc(signal)
/var/folders/5q/565vdp

,major_scales,major_data,minor_scales,minor_data
0,C,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,...",Am,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,..."
1,G,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",Em,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
2,D,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",Bm,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,..."
3,A,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,...",F$^\sharp$m,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,E,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",C$^\sharp$m,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
5,B,"[[0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,...",G$^\sharp$m,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
6,F$^\sharp$,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",D$^\sharp$m,"[[0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,..."
7,D$^\flat$,"[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",B$^\flat$m,"[[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,..."
8,A$^\flat$,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",Fm,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
9,E$^\flat$,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",Cm,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [770]:
# Defining training and testing data 
X = np.array(spike_df.get('major_data'))
X = np.append(X, np.array(spike_df.get('minor_data')))
y = np.tile([1,0],(13,1))
y = np.append(y, np.tile([0,1],(13,1)),axis=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=7)

#labels = sorted(np.unique(y_train))
labels = [[1,0],[0,1]]

dmin = [np.min(X_train[i]) for i in range(X_train.shape[0])]
dmax = [np.max(X_train[i]) for i in range(X_train.shape[0])]



# selecting sample scale 
for i in range(len(X_train)):
    # mel region (20 total) corresponds to id 
    for j in range(len(X_train[i])):
        # time bin selection 
        for k in range(len(X_train[i][j])): 
            if X_train[i][j][k] != 0:
                spike = neuro.Spike(id=j,time=0,value=X_train[i][j][k])
                proc .... 


In [771]:
risp_config = {
  "leak_mode": True,
  "min_weight": -1,
  "max_weight": 1,
  "min_threshold": -1,
  "max_threshold": 1,
  "max_delay": 5
}

proc = risp.Processor(risp_config)

temp_net = neuro.Network()
temp_net.set_properties(proc.get_network_properties())

In [772]:
def create_neuron(neuron_id, net, moa):
    neuron = net.add_node(neuron_id)
    temp_net.randomize_node_properties(moa, neuron)
    return neuron

In [773]:
n_inputs = 20
n_hidden = 40
n_outputs = len(labels)
n_neurons = n_inputs+n_hidden+n_outputs
n_synapses = 100
seed = 42

moa = neuro.MOA()
moa.seed(seed)
random.seed(seed)

In [774]:
if False:
    for i in range(n_inputs):
        neuron = create_neuron(i, temp_net, moa)
        neuron.set("Threshold",0.75)
        temp_net.add_input(i)

    for i in range(n_outputs):
        neuron = create_neuron(i+n_inputs, temp_net, moa)
        neuron.set("Threshold",0.75)
        temp_net.add_output(i)
        
    for i in range(n_hidden):
        neuron = create_neuron(i+n_inputs+n_outputs, temp_net, moa)
        
#if False:
for i in range(n_inputs):
    neuron = create_neuron(i, temp_net, moa)
    neuron.set("Threshold",0.75)
    temp_net.add_input(neuron.id)
    
for i in range(n_outputs):
    neuron = create_neuron(i+n_inputs, temp_net, moa)
    neuron.set("Threshold",0.75)
    temp_net.add_output(neuron.id)
    
for i in range(n_hidden):
    neuron = create_neuron(i+n_inputs+n_outputs, temp_net, moa)




In [775]:
for i in range(n_synapses):
    source = random.randint(0,n_neurons-1)
    dest = random.randint(0,n_neurons-1)
    synapse = temp_net.add_or_get_edge(source, dest)
    temp_net.randomize_edge_properties(moa, synapse)

In [776]:
evolver = eons.EONS(eo_params)
evolver.set_template_network(temp_net)

pop = evolver.generate_population(eo_params,1)

# included into fitness function 
def get_prediction(X):
    proc.clear_activity()
        # selecting sample scale 
    for i in range(len(X_train)):
        # mel region (20 total) corresponds to id 
        for j in range(len(X_train[i])):
            # time bin selection 
            for k in range(len(X_train[i][j])): 
                if X_train[i][j][k] != 0:
                    spike = neuro.Spike(id=j,time=0,value=X_train[i][j][k])
                    proc.apply_spike(spike)
    proc.run(100)
    return labels[proc.output_count_max(n_outputs)[0]]

def fitness(net, X, y):
    proc.load_network(net)
    proc.clear_activity()
    for l in range(net.num_nodes()):
        proc.track_neuron_events(l)
    
        # selecting sample scale 
    for i in range(len(X_train)):
        # mel region (20 total) corresponds to id 
        for j in range(len(X_train[i])):
            # time bin selection 
            for k in range(len(X_train[i][j])): 
                if X_train[i][j][k] != 0:
                    spike = neuro.Spike(id=j,time=0,value=X_train[i][j][k])
                    proc.apply_spike(spike)
        #proc.run(100)
        y_predict = labels[proc.output_count_max(n_outputs)[0]]
        
    
        #y_predict = [get_prediction(x) for x in X]
        return accuracy_score(y_predict, y[i])

In [777]:
def get_prediction(x):
    #print(x)
    #proc.clear_activity()
    for j in range(len(x)):
            # time bin selection 
            for k in range(len(x[j])): 
                if x[j][k] != 0:
                    spike = neuro.Spike(id=j,time=0,value=x[j][k])
                    proc.apply_spike(spike)
    proc.run(50)
    return labels[proc.output_count_max(n_outputs)[0]]

In [778]:
def fitness(net, X, y):
    proc.load_network(net)
    
    # Set up output tracking
    for i in range(n_outputs):
        proc.track_neuron_events(i)
    
    y_predict = [get_prediction(x) for x in X]
    #print(len(y_predict))
    return accuracy_score(y_predict, y)

In [779]:
vals = []
for i in range(100):
    # Calculate the fitnesses of all of the networks in the population
    fitnesses = [fitness(net.network, X_train, y_train) for net in pop.networks]
    
    # Track the best performing network throughout and print the current best result
    max_fit = max(fitnesses)
    mean_fit = np.mean(fitnesses)
    #print(fitnesses)
    vals.append(max_fit)
    print("Epoch ", i, " : ","max fit", max_fit, "mean fit",mean_fit)
    
    # Create the next population based on the fitnesses of the current population
    pop = evolver.do_epoch(pop, fitnesses, eo_params)

Epoch  0  :  max fit 0.5882352941176471 mean fit 0.4205882352941177
Epoch  1  :  max fit 0.5882352941176471 mean fit 0.45000000000000007
Epoch  2  :  max fit 0.5882352941176471 mean fit 0.49764705882352944
Epoch  3  :  max fit 0.5882352941176471 mean fit 0.5229411764705881
Epoch  4  :  max fit 0.5882352941176471 mean fit 0.5458823529411764
Epoch  5  :  max fit 0.5882352941176471 mean fit 0.5458823529411764
Epoch  6  :  max fit 0.5882352941176471 mean fit 0.5494117647058823
Epoch  7  :  max fit 0.5882352941176471 mean fit 0.5423529411764706
Epoch  8  :  max fit 0.5882352941176471 mean fit 0.5405882352941176
Epoch  9  :  max fit 0.5882352941176471 mean fit 0.5294117647058821
Epoch  10  :  max fit 0.5882352941176471 mean fit 0.5617647058823527
Epoch  11  :  max fit 0.5882352941176471 mean fit 0.5517647058823529
Epoch  12  :  max fit 0.5882352941176471 mean fit 0.5529411764705882
Epoch  13  :  max fit 0.5882352941176471 mean fit 0.5617647058823528
Epoch  14  :  max fit 0.9411764705882353 m

In [780]:

best_net = pop.networks[fitnesses.index(max_fit)].network
train = fitness(best_net, X_train, y_train)
print("Training Accuracy: ", train)
test = fitness(best_net, X_test, y_test)
print("Testing Accuracy: ", test)

Training Accuracy:  0.9411764705882353
Testing Accuracy:  0.3333333333333333
